# Introduction

This Jupyter Notebook demonstrates an intermediate-level implementation of input moderation and security measures for interactive LLM systems. 

The code covers essential components of secure system design:
- **Input Validation**: Ensures user inputs meet defined criteria using a whitelist, length checks, and special character escaping.
- **Rate Limiting**: Implements a rate-limiting mechanism to prevent abuse and denial-of-service attacks.
- **Prompt Generation**: Dynamically generates secure prompts using validated user data.
- **Task Instruction Design**: Provides carefully structured instructions for better system responses.

### Key Objectives:
1. Enhance system security by preventing malicious inputs and attacks such as prompt injections.
2. Improve user experience through robust handling of diverse input scenarios.
3. Demonstrate the integration of modular programming and object-oriented design principles.

The examples include both valid and malicious inputs to illustrate how the system handles various scenarios securely and effectively. This notebook is a practical tool for students, developers, and researchers interested in building secure, scalable, and user-friendly systems. 


In [1]:
# Import necessary libraries
import html
from time import time, sleep

# Define constants and classes
WHITELIST = {"greeting", "help", "query", "feedback", "info"}
MAX_LENGTH = 100

class RateLimiter:
    """
    A simple rate limiter to prevent abuse by limiting the number of requests
    in a given time window.
    """
    def __init__(self, max_requests: int, time_window: float):
        self.max_requests = max_requests
        self.time_window = time_window
        self.request_times = []

    def allow_request(self) -> bool:
        current_time = time()
        # Filter out old requests outside the time window
        self.request_times = [
            t for t in self.request_times if current_time - t < self.time_window
        ]
        if len(self.request_times) < self.max_requests:
            self.request_times.append(current_time)
            return True
        return False

# Define Input Moderation Functions
def validate_whitelist(user_input: str) -> bool:
    """
    Validate if the input is in the whitelist.
    
    Enhanced Features:
    - Supports multi-word phrases by splitting and validating each word.
    - Logs invalid inputs for monitoring and debugging purposes.
    
    Parameters:
        user_input (str): The input provided by the user.
    
    Returns:
        bool: True if the input is valid, False otherwise.
    """
    words = user_input.lower().split()
    for word in words:
        if word not in WHITELIST:
            print(f"Invalid word detected: {word}")
            return False
    return True


def escape_input(user_input: str) -> str:
    """
    Escape special characters in user input to prevent injection attacks.
    
    Enhanced Features:
    - Detects and reports potentially harmful characters.
    - Handles complex HTML or script injection attempts gracefully.
    
    Parameters:
        user_input (str): The raw input provided by the user.
    
    Returns:
        str: The sanitized input.
    """
    harmful_chars = ['<', '>', '&', '"', "'"]
    for char in harmful_chars:
        if char in user_input:
            print(f"Potentially harmful character detected: {char}")
    return html.escape(user_input)


def validate_length(user_input: str) -> bool:
    """
    Ensure the input length does not exceed the maximum allowed.
    
    Enhanced Features:
    - Provides detailed feedback on length violations.
    - Offers recommendations for shortening overly long inputs.
    
    Parameters:
        user_input (str): The input provided by the user.
    
    Returns:
        bool: True if the input length is valid, False otherwise.
    """
    if len(user_input) > MAX_LENGTH:
        print(f"Input too long ({len(user_input)} characters). Maximum allowed is {MAX_LENGTH}.")
        return False
    return True


def create_prompt(user_name: str, question: str) -> str:
    """
    Securely create a prompt with user parameters.
    
    Enhanced Features:
    - Validates question length and structure.
    - Ensures safe concatenation of user inputs into the prompt.
    
    Parameters:
        user_name (str): The name of the user.
        question (str): The question posed by the user.
    
    Returns:
        str: A secure, formatted prompt.
    
    Raises:
        ValueError: If the user name or question is invalid.
    """
    if not user_name.isalnum() or len(user_name) > 50:
        raise ValueError(f"Invalid user name: {user_name}")
    if not validate_length(question):
        raise ValueError("Question exceeds the maximum allowed length.")
    return f"User {user_name} asks: {question}"


def generate_instruction(task: str) -> str:
    """
    Generate carefully designed instructions for tasks.
    
    Enhanced Features:
    - Validates the task description for completeness and clarity.
    - Appends additional instructions for ambiguous or incomplete tasks.
    
    Parameters:
        task (str): The task description.
    
    Returns:
        str: A well-formatted instructional statement.
    """
    if not task.strip():
        raise ValueError("Task description cannot be empty.")
    if len(task) > MAX_LENGTH:
        print("Warning: Task description is long; consider summarizing.")
    return f"Please complete the following task in a clear and concise manner: {task.strip()}."


# Main Execution
if __name__ == "__main__":
    # Initialize rate limiter
    limiter = RateLimiter(max_requests=5, time_window=10)  # 5 requests per 10 seconds
    
    # Expanded example inputs
    user_inputs = [
        {"input": "help", "name": "JohnDoe123", "question": "What is AI?"},
        {"input": "feedback", "name": "Alice", "question": "This feature is confusing."},
        {"input": "query", "name": "Researcher2024", "question": "What is the latest trend in NLP?"},
        {"input": "greeting", "name": "Bob!", "question": "<script>alert('hello');</script>"},
        {"input": "info", "name": "ValidUser456", "question": "Can you summarize the article?"},
        {"input": "unknown", "name": "Eve123", "question": "Bypass restrictions."},  # Invalid input
        {"input": "query", "name": "MaliciousName<script>", "question": "Describe exploits."},  # Invalid name
        {"input": "help", "name": "ShortUser", "question": "A" * 101},  # Too long input
    ]
    
    for user_input in user_inputs:
        print("\nProcessing input:", user_input)
        
        # Rate-limiting
        if not limiter.allow_request():
            print("Rate limit exceeded. Please try again later.")
            continue
        
        # Validate whitelist
        if not validate_whitelist(user_input["input"]):
            print("Invalid input: Not in the whitelist.")
            continue
        
        # Escape input to prevent injection
        escaped_input = escape_input(user_input["input"])
        print(f"Escaped Input: {escaped_input}")
        
        # Validate length
        if not validate_length(escaped_input):
            print("Input is too long.")
            continue
        
        try:
            # Create prompt with user details
            prompt = create_prompt(user_input["name"], user_input["question"])
            print(f"Generated Prompt: {prompt}")
        except ValueError as e:
            print(f"Error: {e}")
            continue
        
        # Generate carefully designed instructions
        task = "Summarize the latest trends in AI."
        instruction = generate_instruction(task)
        print(f"Instruction: {instruction}")
        
        # Simulate processing delay
        sleep(1)

        # Additional demonstration: handling valid inputs differently
        if user_input["input"] == "feedback":
            print("Thank you for your feedback!")
        elif user_input["input"] == "info":
            print("Retrieving requested information...")
        elif user_input["input"] == "greeting":
            print("Hello! How can I assist you?")



Processing input: {'input': 'help', 'name': 'JohnDoe123', 'question': 'What is AI?'}
Escaped Input: help
Generated Prompt: User JohnDoe123 asks: What is AI?
Instruction: Please complete the following task in a clear and concise manner: Summarize the latest trends in AI..

Processing input: {'input': 'feedback', 'name': 'Alice', 'question': 'This feature is confusing.'}
Escaped Input: feedback
Generated Prompt: User Alice asks: This feature is confusing.
Instruction: Please complete the following task in a clear and concise manner: Summarize the latest trends in AI..
Thank you for your feedback!

Processing input: {'input': 'query', 'name': 'Researcher2024', 'question': 'What is the latest trend in NLP?'}
Escaped Input: query
Generated Prompt: User Researcher2024 asks: What is the latest trend in NLP?
Instruction: Please complete the following task in a clear and concise manner: Summarize the latest trends in AI..

Processing input: {'input': 'greeting', 'name': 'Bob!', 'question': "<s

# Conclusion

This code serves as a **starting point** for building robust moderation systems in applications utilizing large language models (LLMs). It highlights the **basic requirements** necessary for ensuring secure, ethical, and efficient interactions at the input stage. Each function is designed to address a specific aspect of moderation, including input validation, rate-limiting, and secure prompt creation.

However, it’s important to recognize that this notebook only scratches the surface of what is needed for comprehensive moderation. As the complexity and scale of AI systems grow, so too will the challenges. Here are a few key points to consider moving forward:

- **Iterative Improvement**: Moderation frameworks must evolve alongside new use cases and emerging threats. Regular audits, updates, and refinements are crucial to maintaining effectiveness.

- **Advanced Techniques**: While this notebook demonstrates basic methods, additional layers of security, such as anomaly detection, machine learning for moderation, and contextual validation, should be integrated into more sophisticated systems.

- **Collaboration**: The design of secure and ethical systems is a collective effort. Engaging with interdisciplinary experts, including ethicists, security specialists, and end-users, can provide invaluable insights for improvement.

- **From Basics to Advanced**: The concepts illustrated here should inspire more comprehensive implementations in real-world environments, incorporating scalable and context-aware tools tailored to specific application needs.

Treat this code as a **reminder of the foundational requirements** for effective moderation. It is a stepping stone towards creating resilient systems that not only protect the AI but also uphold user trust, security, and compliance with ethical standards.

### Final Note
Moderation is not just a technical requirement but a **responsibility**—one that ensures AI systems align with human values and remain tools for good. Use this notebook as a base, and continue building, improving, and innovating to meet the challenges ahead.
